In [1]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib as plt
import geopandas
import random
from osgeo import ogr,gdal

Creo un geodataset con las coordenadas de cada piscina

In [2]:
gdf = geopandas.GeoDataFrame.from_file(r'C:\Users\Usuario\Desktop\QGIS-Pool\Git\Pool_shapefile\Pool.shp')
for row in gdf.iterrows():
    gdf['X']=gdf.geometry.x
    gdf['Y']=gdf.geometry.y

Creo una imagen alrededor de cada piscina y guardo las coordenadas de todas las que aparecen

In [3]:
#Defino los parámetros
InputImage = r'C:\Users\Usuario\Desktop\QGIS-Pool\Ortofotos\PNOA_MA_OF_ETRS89_HU31_h50_0823.tif'
Shapefile = r'C:\Users\Usuario\Desktop\QGIS-Pool\Git\Pool_shapefile\Pool.shp'
RasterFormat = 'GTiff'
PixelRes = 0.5
VectorFormat = 'ESRI Shapefile'
folder = 'C:\\Users\\Usuario\\Desktop\\QGIS-Pool\\Git\\images\\'
tilemapping = {}

Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
Projection = Raster.GetProjectionRef()

Count = 0
for index, row in gdf.iterrows():
    Count +=1
    xmid = row['X'] + random.randint(-40, 40)  #Desplazo el centro de la imagen de forma aleatoria
    ymid = row['Y'] + random.randint(-40, 40)
    xmin, ymin = xmid - 50, ymid - 50  #Creo el extent de la imagen
    xmax, ymax = xmid + 50, ymid + 50
    
    ImageName = str(Count) + '.tif'
    OutTileName = folder + ImageName
    OutTile = gdal.Translate(OutTileName, Raster, width=224, height=224, projWin = [xmin, ymax, xmax, ymin]) #Guardo la imagen
    OutTile = None
    
    #Guardo en un diccionario las coordenadas de todas las piscinas que aparecen en la foto actual
    centers = []
    for index, row in gdf.iterrows():
        x = row['X']
        y = row['Y']
        if (x < xmax) & (x > xmin) & (y < ymax) & (y > ymin):
            x = (x - xmin) / (xmax - xmin)
            y = (y - ymin) / (ymax - ymin)
            centers.append((x,y))
            
    if len(centers) > 0:
        tilemapping[ImageName] = centers
        
        
print("Done.")

Done.


Creo un Pickle del diccionario para almacenarlo

In [4]:
import pickle
pickle.dump(tilemapping, open(r'C:\Users\Usuario\Desktop\QGIS-Pool\Git\tilemapping.pkl', 'wb'))